# Histogram of Oriented Gradients (HOG) with CIFAR10 Dataset with `pytorch`

In [ ]:
import torch
from torchvision.datasets import CIFAR10
from torchvision import transforms

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
cifar_train = CIFAR10("./data_cifar10/", download = True, transform = transforms.ToTensor())
cifar_test = CIFAR10("./data_cifar10/", train = False, download = True, transform = transforms.ToTensor())